In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [11]:
# Configuração dos parâmetros do banco
host='localhost'
port=3306
database='constructal_automate_results'
user='constructal_automate'
password='constructal_design_2024'

query = f'''
SELECT 
    sp.N_ls, 
    sp.N_ts, 
    sp.k, 
    sp.h_s, 
    sp.t_s, 
    (p.b + sp.N_ls * sp.h_s) AS L, 
    (p.b * sp.t_1 + sp.N_ls * sp.h_s * sp.t_s) AS A,
    ROUND((epb1.n_u * (p.b + sp.N_ls * sp.h_s) / (p.b * sp.t_1 + sp.N_ls * sp.h_s * sp.t_s)),2) as TENSAO_1A,
    ROUND((epb1.n_u * (p.b + sp.N_ls * sp.h_s) / (p.b * sp.t_1 + sp.N_ls * sp.h_s * sp.t_s)) / 186.46, 3) AS NUS_1A,
    ROUND((epb2.n_u /sp.t_1), 2) AS TENSAO_2A,
    ROUND((epb2.n_u /sp.t_1) / 59.68, 3) AS NUS_2A
FROM constructal_automate_results.csg_stiffenedplate AS sp
INNER JOIN constructal_automate_results.cbeb_stiffenedplateanalysis AS spa1 
    ON spa1.stiffened_plate_id = sp.id AND spa1.buckling_load_type_id = 1
INNER JOIN constructal_automate_results.cbeb_elastoplasticbuckling AS epb1 
    ON epb1.stiffened_plate_analysis_id = spa1.id
INNER JOIN constructal_automate_results.cbeb_stiffenedplateanalysis AS spa2 
    ON spa2.stiffened_plate_id = sp.id AND spa2.buckling_load_type_id = 2
INNER JOIN constructal_automate_results.cbeb_elastoplasticbuckling AS epb2  
    ON epb2.stiffened_plate_analysis_id = spa2.id and epb2.id not between 259 and 266
INNER JOIN constructal_automate_results.csg_plate AS p 
    ON p.id = sp.plate_id;
'''

base_path = 'tabelas_dissertacao/'

In [3]:
file_name = 'estudo_comparativo_lima_2016_com_tensao'
csv_name = file_name + ".csv"
latex_name = file_name + ".tex"

In [4]:
def retrieve_data_from_db(host, port, user, password, database, query):
    try:
        # Criar a conexão com o banco de dados
        connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
        engine = create_engine(connection_string)

        # Executar a query e carregar os dados em um DataFrame
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"Erro: {e}")

In [12]:
df_results = retrieve_data_from_db(host, port, user, password, database, query)

In [13]:
df_results

,N_ls,N_ts,k,h_s,t_s,L,A,TENSAO_1A,NUS_1A,TENSAO_2A,NUS_2A
0,0,0,0.000,0.0,0.0,1000.0,14000.0,186.46,1.000,59.68,1.000
1,2,2,56.200,281.0,5.0,1562.0,12610.0,31.03,0.166,25.57,0.428
2,2,2,14.100,141.0,10.0,1282.0,12620.0,352.90,1.893,196.18,3.287
3,2,2,6.333,95.0,15.0,1190.0,12650.0,325.50,1.746,211.45,3.543
4,2,2,3.550,71.0,20.0,1142.0,12640.0,312.62,1.677,219.08,3.671
...,...,...,...,...,...,...,...,...,...,...,...
124,5,5,1.450,29.0,20.0,1145.0,12700.0,290.49,1.558,120.23,2.015
125,5,5,0.960,24.0,25.0,1120.0,12800.0,265.50,1.424,100.37,1.682
126,5,5,0.666,20.0,30.0,1100.0,12800.0,236.84,1.270,87.20,1.461
127,5,5,0.485,17.0,35.0,1085.0,12775.0,213.55,1.145,82.18,1.377


In [45]:
df_from_lima_2016 = pd.read_excel('resultados_estudo_comparativo.xlsx', sheet_name='send_to_pandas')

In [23]:
df_from_lima_2016

,N_ls,N_ts,k,NUS_1A_LIMA_2016
0,0,0,0.000,1.000000
1,2,2,56200.000,0.489126
2,2,2,14100.000,1.885194
3,2,2,6333.000,1.743734
4,2,2,3550.000,1.670048
...,...,...,...,...
124,5,5,1450.000,1.514864
125,5,5,0.960,1.312208
126,5,5,0.666,1.232842
127,5,5,0.485,1.141350


In [46]:
df_results['NUS_1A_LIMA_2016'] = df_from_lima_2016['NUS_1A_LIMA_2016'].round(3)

In [29]:
pd.set_option('display.max_rows', None)

In [10]:
df_results

,SP_ID,N_ls,N_ts,k,h_s,t_s,L,A,TENSAO_1A,NUS_1A,TENSAO_2A,NUS_2A
0,1,0,0,0.000,0.0,0.0,1000.0,14000.0,186.46,1.000,59.68,1.000
1,2,2,2,56.200,281.0,5.0,1562.0,12610.0,31.03,0.166,25.57,0.428
2,3,2,2,14.100,141.0,10.0,1282.0,12620.0,352.90,1.893,196.18,3.287
3,4,2,2,6.333,95.0,15.0,1190.0,12650.0,325.50,1.746,211.45,3.543
4,5,2,2,3.550,71.0,20.0,1142.0,12640.0,312.62,1.677,219.08,3.671
...,...,...,...,...,...,...,...,...,...,...,...,...
124,125,5,5,1.450,29.0,20.0,1145.0,12700.0,290.49,1.558,120.23,2.015
125,126,5,5,0.960,24.0,25.0,1120.0,12800.0,265.50,1.424,100.37,1.682
126,127,5,5,0.666,20.0,30.0,1100.0,12800.0,236.84,1.270,87.20,1.461
127,128,5,5,0.485,17.0,35.0,1085.0,12775.0,213.55,1.145,82.18,1.377


In [48]:
df_results['Dif. (%)'] = abs((df_results['NUS_1A_LIMA_2016'] - df_results['NUS_1A_LANCANOVA_2024']) / df_results['NUS_1A_LIMA_2016']) * 100

In [49]:
media_dif = df_results['Dif. (%)'].mean()
print(media_dif)

11.339874517466635


In [51]:
df_sem_flambagem_local = df_results.drop(index=[1,9,17,25,33,41,49,57,65,73,81,89,97,105,113,121])

In [52]:
media_dif_sfl = df_sem_flambagem_local['Dif. (%)'].mean()
print(media_dif_sfl)

5.287638605727319


In [53]:
df_results.to_excel('resultados_verificacao_software.xlsx', index=False)

In [5]:
def general_formatter(fmt, value):
    return f'{fmt}' % value

def formatter_d(value):
    return '%d' % value

def formatter_float1(value):
    return '%.1f' % value

def formatter_float2(value):
    return '%.2f' % value

def formatter_float3(value):
    return '%.3f' % value

def formatter_float4(value):
    return '%.4f' % value

In [6]:
def generate_latex_from_df(df):
    # Salvar o resultado como um arquivo CSV
    df.to_csv(base_path + csv_name, index=False)

    formatters = ['%d', '%d']

    # Converter o DataFrame para LaTeX
    print(f"Convertendo o arquivo CSV para LaTeX")
    latex_table = df.to_latex(formatters=[formatter_d, formatter_d, formatter_float3, formatter_d, formatter_d, formatter_d, formatter_d, formatter_float2, formatter_float3, formatter_float3],
                              decimal=',',
                              index=False)

    # Salvar a tabela LaTeX em um arquivo .tex
    with open(base_path + latex_name, "w") as f:
        f.write(latex_table)

    print("Tabela LaTeX criada com sucesso!")

In [56]:
df_5_5 = df_results.loc[[121,122,123,124,125,126,127,128]]

In [57]:
df_5_5

,N_ls,N_ts,k,h_s,t_s,L,A,NUS_1A_LANCANOVA_2024,NUS_1A_LIMA_2016,Dif. (%)
121,5,5,22.600,113.0,5.0,1565.0,12625.0,0.930,1.147,18.918919
122,5,5,5.700,57.0,10.0,1285.0,12650.0,1.817,1.796,1.169265
123,5,5,2.600,39.0,15.0,1195.0,12725.0,1.727,1.687,2.371073
124,5,5,1.450,29.0,20.0,1145.0,12700.0,1.558,1.515,2.838284
125,5,5,0.960,24.0,25.0,1120.0,12800.0,1.424,1.312,8.536585
126,5,5,0.666,20.0,30.0,1100.0,12800.0,1.270,1.233,3.000811
127,5,5,0.485,17.0,35.0,1085.0,12775.0,1.145,1.141,0.350570
128,5,5,0.375,15.0,40.0,1075.0,12800.0,1.124,1.078,4.267161


In [61]:
generate_latex_from_df(df_5_5)

Convertendo o arquivo CSV para LaTeX
Tabela LaTeX criada com sucesso!


In [9]:
generate_latex_from_df(df_results)

Convertendo o arquivo CSV para LaTeX


IndexError: list index out of range